In [2]:
import random
import time
import copy

# Quick hack
import sys
sys.path.append('./source')

# Import symbolic execution
from MT19937 import MT19937, MT19937_symbolic

# Import XorSolver
from XorSolver import XorSolver

## Consistency between MT19937 and MT19937_symbolic classes


In [3]:
# Init MT199937 symbolic
rng_sym = MT19937_symbolic()

# Init state: list of 624 32 bit numbers, as per MT19937
state = list(random.getstate()[1][:-1])

# Init MT19937 with state
rng = MT19937(state.copy())

t = time.time()

for _ in range(1000):
    
    # Get random number n
    n = rng()
    
    # Get symbolic representation of n
    n_sym = rng_sym()
    
    # Check symbolic execution is correct
    assert n == rng_sym.cast_num(state, n_sym)
    
print("Time taken: {}s".format(time.time() - t))

# > Time taken: 2.27s with cast_num
# > Time taken: 0.12s without cast_num

Time taken: 1.5479977130889893s


## MT19937 Cloning of Python's random library getrandbits(4) with precomputed matrix

Precomputed matrix only support nbits=[4,8,12,16,20,24,28,32]. Code below only uses only nbits=4 bits of MT19937 output

In [6]:
from math import ceil
# Init python's getrandbits with 4 bits
nbits = 1
rng = lambda: random.getrandbits(nbits)

# Collecting data
eqns = []
n_test = []
itrr = ceil(19968/nbits)
itrr = 2492
# itrr = 624*int((31.9//nbits)+1)

def readNdata():
    ndata = {}
    with open('record.txt') as f:
        for i in f.readlines():
            nb,nd = map(int,i.split(','))
            ndata[nb] = nd
    return ndata

ndata = readNdata()[nbits]
for _ in range(ndata):
    # Get random number from rng and save for later testing
    n = rng()
    n_test.append(n)

t = time.time()

# Cloning MT19937 from data
rng_clone = MT19937(state_from_data = (n_test, nbits),useMyMat=2)

print("Time taken: {}s".format(time.time() - t))

# Test if cloning has been successful
for n in n_test:
    assert n == rng_clone() >> (32-nbits), "Clone failed!"



print("[*] Cloning successful!")

useMyMat = 2
inv_filename = 'myMat1.npy.gz'
nbits = 1,verify Failed, [Errno 2] No such file or directory: 'd:\\Desktop\\毕设\\MT19937-Symbolic-Execution-and-Solver\\./source/../matrices/verify_nb-1_iterr-19968_ndata-12460032.npy.gz'
Time taken: 87.1941146850586s
[*] Cloning successful!


In [2]:
def test(nbits,useMyMat):
    rng = lambda: random.getrandbits(nbits)

    # Collecting data
    eqns = []
    n_test = []
    for _ in range(624*int((31.9//nbits)+1)):
        # Get random number from rng and save for later testing
        n = rng()
        n_test.append(n)

    t = time.time()

    # Cloning MT19937 from data
    rng_clone = MT19937(state_from_data = (n_test, nbits),useMyMat=useMyMat)

    t = round(time.time() - t,3)
    # print("Time taken: {}s".format(t))

    # Test if cloning has been successful
    for n in n_test:
        if not n == rng_clone() >> (32-nbits):
            print(f"[*] {nbits}-bits Cloning Failed")
            return -1
        
    print(f"[*] {nbits}-bits Cloning successful!")
    return t

nbits = 32
try:
    t1 = [test(nbits,0) for _ in range(5)]
except:
    t1 = [1]
t2 = [test(nbits,2) for _ in range(3)]
avg = lambda x:sum(x)/len(x)
print(t1,t2)
print(f"{nbits}-bits,avg:{avg(t1)},{avg(t2)}")

useMyMat = 0
[*] 32-bits Cloning successful!
useMyMat = 0
[*] 32-bits Cloning successful!
useMyMat = 0
[*] 32-bits Cloning successful!
useMyMat = 0
[*] 32-bits Cloning successful!
useMyMat = 0
[*] 32-bits Cloning successful!
useMyMat = 0
[*] 32-bits Cloning successful!
useMyMat = 0
[*] 32-bits Cloning successful!
useMyMat = 0
[*] 32-bits Cloning successful!
useMyMat = 0
[*] 32-bits Cloning successful!
useMyMat = 0
[*] 32-bits Cloning successful!
useMyMat = 2
[*] 32-bits Cloning successful!
useMyMat = 2
[*] 32-bits Cloning successful!
useMyMat = 2
[*] 32-bits Cloning successful!
[0.114, 0.079, 0.101, 0.109, 0.074] [0.105, 0.119, 0.073]
32-bits,avg:0.09540000000000001,0.09899999999999999


## MT19937 Cloning of Python's random library getrandbits(32) using Python-only solver

Uses all 32 bits of MT19937

In [8]:
### Creating the equations to solve

# Init python's getrandbits with 32 bits
nbits = 16
print(f"{nbits = }")
rng = lambda: random.getrandbits(nbits)

# Init MT199937 symbolic
rng_sym = MT19937_symbolic()

# Build system of equations
eqns = []
n_test = []
# for _ in range(624):
for _ in range(624*int((31.9//nbits)+1)):
    
    # Get random number from rng and save for later testing
    n = rng()
    n_test.append(n)
    
    # Split n into binary (A list of bools)
    eqn_rhs_list = rng_sym._int2bits(n)
    
    # Get symbolic representation of n
    eqn_lhs_list = rng_sym()[:nbits]
    
    # Add to eqns
    for lhs,rhs in zip(eqn_lhs_list, eqn_rhs_list):
        eqns.append([lhs,rhs])

### Using the python only solver XorSolver.solve

nvars = 624*32
nvars = nbits*624*int((31.9//nbits)+1)
eqns_copy = copy.deepcopy(eqns)

# Initialise solver with eqns
solver = XorSolver(eqns_copy, nvars)

t = time.time()

# Solve eqns. Takes aroung 100s to solve
# verbose=False to suppress output
solver.solve(verbose=True)

print("Time taken to solve: {}s".format(time.time() - t))

# Clone MT19937 with solver.eqns (fully solved by now)
rng_clone = MT19937(state_from_solved_eqns = solver.eqns)

# Test if cloning has been successful
for n in n_test:
    assert n == rng_clone()>>(32-nbits), "Clone failed!"
    
print("[*] Cloning successful!")

nbits = 16


100%|██████████| 19968/19968 [01:41<00:00, 195.98it/s]

Time taken to solve: 101.90407299995422s


AssertionError: Clone failed!

In [10]:
rng_clone()

59559

## MT19937 Cloning of Python's random library getrandbits(16) using cryptominisat

Using Python wrapper of cryptominisat available on XorSolver. Uses only 16 bits of MT19937 output

In [6]:
### Creating the equations to solve

# Init python's getrandbits with 16 bits
nbits = 16
rng = lambda: random.getrandbits(nbits)

# Init MT199937 symbolic
rng_sym = MT19937_symbolic()

# Build system of equations
eqns = []
n_test = []
for _ in range(624*32//nbits):
    
    # Get random number from rng and save for later testing
    n = rng()
    n_test.append(n)
    
    # Split n into binary (A list of bools) and only take 8 bits
    eqn_rhs_list = rng_sym._int2bits(n, bits=nbits)
    
    # Get symbolic representation of n
    eqn_lhs_list = rng_sym()
    
    # Add to eqns
    for lhs,rhs in zip(eqn_lhs_list, eqn_rhs_list):
        eqns.append([lhs,rhs])

In [7]:
### Using XorSolver's cryptominisat wrapper

nvars = 624*32
eqns_copy = copy.deepcopy(eqns)

# Initialise solver with eqns
solver = XorSolver(eqns_copy, nvars)

t = time.time()

# Solve eqns, takes about 250s without m4ri
cryptominisat_path = r".\bin\cryptominisat5_win10_x64_no-m4ri.exe"
cnf_file = "test.cnf"
solver.cryptominisat_solve(cryptominisat_path, cnf_file, verbose=0) # Disable all output with verbose=0

print("Time taken to solve: {}s".format(time.time() - t))

# Clone MT19937 with solver.eqns (fully solved by now)
rng_clone = MT19937(state_from_solved_eqns = solver.eqns)

# Test if cloning has been successful
for n in n_test:
    assert n == rng_clone() >> (32-nbits), "Clone failed!"
    
print("[*] Cloning successful!")

Time taken to solve: 245.00420260429382s
[*] Cloning successful!


## Reversing MT19937 State with MT19937.reverse_states

A functionality in progress. There are still bugs with it. For now it seems to work sometimes. In the event that it fails, simply choose a different chunk of data and reverse the state from there.

In [30]:
# Initialise rng (random.getrandbits)
rng = lambda: random.getrandbits(32)
[rng() for _ in range(11174)]
# Increment state of random object (random.getstate()) 10000 times
# Save output for testing later
n_test = []
shift = 10000
for _ in range(shift):
    n = rng()
    n_test.append(n)

# save next 624 numbers for cloning
data = []
for _ in range(624):
    n = rng()
    n_test.append(n)
    data.append(n)
    
# rng's state has incremented 10000 times before we got the data we need to clone
# We want to recover rng's original state by 
# cloning rng's state after it was incremented 10000 times and
# Reversing the state by 10000 steps

# Clone rng's current state
rng_clone = MT19937(state_from_data = (data, 32))

# Reverse state by 10000
rng_clone.reverse_states(shift)

# Test if cloning has been successful
# Does not work all the time
n_wrong = []
for idx, n in enumerate(n_test):
    if rng_clone() != n:
        n_wrong.append(idx)
if len(n_wrong) == 0:
    print("Reversing successful!")
else:
    print("Reversing failed. Predicted {}/{} numbers wrong".format(len(n_wrong), len(n_test)))

Reversing failed. Predicted 500/10624 numbers wrong


In [31]:
n_wrong

[27,
 28,
 29,
 30,
 31,
 32,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 431,
 432,
 433,
 434,
 435,
 436,
 437,
 438,
 488,
 489,
 490,
 491,
 492,
 493,
 494,
 495,
 496,
 542,
 543,
 544,
 545,
 546,
 547,
 548,
 549,
 650,
 651,
 652,
 653,
 654,
 655,
 658,
 659,
 660,
 661,
 662,
 663,
 664,
 665,
 769,
 770,
 771,
 772,
 773,
 774,
 775,
 776,
 827,
 828,
 829,
 830,
 831,
 832,
 833,
 834,
 877,
 878,
 879,
 880,
 881,
 882,
 885,
 886,
 887,
 888,
 889,
 890,
 891,
 892,
 1104,
 1105,
 1106,
 1107,
 1108,
 1109,
 1166,
 1167,
 1168,
 1169,
 1170,
 1171,
 1172,
 1274,
 1275,
 1276,
 1277,
 1278,
 1281,
 1282,
 1283,
 1284,
 1285,
 1286,
 1287,
 1288,
 1331,
 1332,
 1333,
 1334,
 1335,
 1336,
 1450,
 1451,
 1452,
 1453,
 1454,
 1455,
 1456,
 1457,
 1558,
 1559,
 1560,
 1561,
 1562,
 1563,
 1677,
 1678,
 1679,
 1680,
 1681,
 1682,
 1683,
 1684,
 1728,
 1729,
 1730,
 1731,
 1732,
 

In [13]:
## If the above fails, forward rng_clone a few states and reverse

rng_clone = MT19937(state_from_data = (data, 32))

offset = random.randint(0, 624*10)
print('Offset used:', offset)
for _ in range(offset):
    rng_clone()

rng_clone.reverse_states(shift + offset)

# Test if cloning has worked.
# Might have to repeat a few times for it to work
n_wrong = []
for idx, n in enumerate(n_test):
    if rng_clone() != n:
        n_wrong.append(idx)
if len(n_wrong) == 0:
    print("Reversing successful!")
else:
    print("Reversing failed. Predicted {}/{} numbers wrong".format(len(n_wrong), len(n_test)))

Offset used: 4229
Reversing successful!
